### Запуск QtLab

In [1]:
%run D:\qtlab_replacement\init.py
%matplotlib qt4
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *

In [2]:
#Параметры кубитов.
Qubits = {1: {'Fr': 4.242e9,'F01': 6.764e9, 'I0': 0.,'Ispan': 0.,'P': -40,'Pex': -2,'Span': 200e6},
          2: {'Fr': 5.8075e9,'F01': 6.764e9, 'I0': 0.,'Ispan': 0.,'P': -40,'Pex': -2,'Span': 200e6},
          3: {'Fr': 6.586e9,'F01': 6.764e9, 'I0': 0.,'Ispan': 0.,'P': -40,'Pex': -2,'Span': 200e6},
          4: {'Fr': 7.5216e9,'F01': 6.764e9, 'I0': 0.,'Ispan': 0.,'P': -40,'Pex': -2,'Span': 200e6}}



# Спектроскопия

In [3]:
pna = Agilent_N5242A('pna', address = 'PNA')
lo1 = Agilent_E8257D('lo1', address = 'PSG1')

In [4]:
current_src = Yokogawa_GS210(address = 'GPIB::2::INSTR')

In [5]:
current_src.get_status()

1

In [6]:
# current_src2 = Yokogawa_GS210(address = 'GPIB::1::INSTR')

In [7]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

True

## Резонаторная спектроскопия

In [ ]:
pna.set_power(-45)
pna.set_nop(1001)
pna.set_xlim(7.7e9, 7.74e9)
pna.set_bandwidth(100)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()

In [ ]:
# current_src.set_range(1e-1)

In [ ]:
current_src.get_current()

In [69]:
res_l = np.array([4.2302, 4.391, 4.528, 4.673, 4.822, 4.999, 5.150, 5.301, 5.450, 5.603, 5.807, 5.912, 6.055, 6.185, 6.226, 6.367, 6.514, 6.662, 6.694, 6.965, 7.124, 7.29, 7.405, 7.70, 8.4])
res_l[9]

5.6029999999999998

In [70]:
res_r = np.array([4.2304, 4.3915, 4.532, 4.709, 4.858, 4.9995, 5.153, 5.304, 5.455, 5.607, 5.810, 5.916, 6.091, 6.195, 6.284, 6.371, 6.545, 6.703, 6.793, 6.970, 7.131,  7.295,7.436, 7.74, 8.42])

In [71]:
len(res_r)

25

In [92]:
i_res = 22
current_max = 0.1e-3
current_nops = 51
freq_nops = 101
power_level = -50

In [93]:
lo1.set_status(False)
currents= np.linspace(-current_max, current_max, current_nops)
#freqs = np.linspace(6.585e9,6.587e9,51)
pna.set_nop(freq_nops)
pna.set_bandwidth(100)
pna.set_power(power_level)
current_src.set_status(1)
pna.set_xlim(res_l[i_res]*1e9, res_r[i_res]*1e9)
result = sweep.sweep(pna, (currents, current_src.set_current, 'Coil current'), filename='anticrossings Rev_6.5')

Started at:  Oct 20 2017 15:19:55


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 20.02 s,	parameter values: Coil current: 4.8e-05,	average cycle time:  1.5s	s	

KeyboardInterrupt: 

In [ ]:
pna.set_xlim(6.580e9, 6.590e9)
pna.set_nop(51)

In [ ]:
powers = np.linspace(-65, 5, 15)
sweep.sweep(pna, (powers, pna.set_power, 'readout power'), filename='Resonator power spectroscopy')

## Кубитная спектроскопия

In [ ]:
tone2_power=-20
frequency_min=Qubits[3]['Fr']-50e6
frequency_max=Qubits[3]['Fr']+10e6
step_freq=301
step_curr=301
current_min=0.0025
current_max=-0.0025
tone1_power=-30
tone1_freq=6.5867e9
freq=np.linspace(frequency_min, frequency_max, step_freq)
curr=np.linspace(current_min, current_max, step_curr)
resolution_bw=200

In [ ]:
pna.set_power(tone1_power)
pna.set_xlim(tone1_freq, tone1_freq)
pna.set_nop(1)
pna.set_bandwidth(resolution_bw)
lo1.set_power(tone2_power)
lo1.set_status(True)

In [ ]:
sweep.sweep(pna, (curr, current_src.set_current,'current'), (freq, lo1.set_frequency, '2nd tone frequency'), filename='2nd tone')

In [ ]:

sweep.sweep(pna, (curr, current_src.set_current,'current'), filename='2nd tone')

In [ ]:
#Кубит
q_id = 1

pna.set_average(0)
pna.set_averages(1)
pna.set_power(Qubits[q_id]['P'])
pna.set_nop(1)
pna.set_centerfreq(Qubits[q_id]['Fr'])
pna.set_bandwidth(5)

lo1.set_power(Qubits[q_id]['Pex'])
lo1.set_status(1)

### 1D скан при фиксированной мощности

In [ ]:
pna.set_nop(1)
pna.set_xlim(5.8e9, 5.8e9)

In [ ]:
#Измерение
lo1.set_status(True)
ex_freqs = np.arange(2.8e9, 3e9, 5e6)

pna.set_bandwidth(5)
pna.set_power (-55)
lo1.set_power (10)
#current_src.set_current(-2.5e-3)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "ex_{:.1f}dBm".format(lo1.get_power()))
except KeyboardInterrupt: pass 

### 2D скан (мощность второго тона, частота второго тона)

In [ ]:
ex_freqs = np.arange(5.5e9, 7e9, 10e6)
ex_powers = np.arange(-20, 15, 1)

pna.set_power(-50)
pna.set_bandwidth(10)
#lo1.set_power(-20)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Second tone power'), (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "#2{:.1f}dBm".format(pna.get_power()),\
           header = header)
lo1.set_status(0)

# Импульсы

### Создаём девайсы

In [ ]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'PXA')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = pna
lo_ro = Labbrick(name='lo_ro', serial=15257)
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')

### Загружаем импульсные скрипты всякие

In [ ]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)

### Осцилляторы

In [ ]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
lo_ex_pow = 10
#lo_ex = lo1
#lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
lo_ro_pow = 10
lo_ro.set_power(lo_ro_pow)

lo_ex_pow = 16
lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)

### AWGшки

In [ ]:
# промежуточные частоты для гетеродинной схемы:
ex_if = 125e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 2В.
for channel in range(1,5):
    awg_tek.set_amplitude(1., channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 1, 2, lo_ex)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(Qubits[1]['F01'])
iq_ro.set_frequency(Qubits[1]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-1, channel=1)
awg_tek.set_marker2_low(-1, channel=1)
awg_tek.set_marker1_high(2, channel=1)
awg_tek.set_marker2_high(2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды
ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)

### Оцифровщик

In [ ]:
# настройки оцифровщика
adc.set_clock(ro_clock)
adc.reset()            
adc.set_timeout(10000)


#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())

### Калибровка миксеров

In [ ]:
#sa.set_ref(0)
iq_ex.get_calibration(sa)
iq_ro.get_calibration(sa)

### Длительность и амплитуда считывающих импульсов

In [ ]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*3/4
    
    print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [ ]:
ro_adc_length = 0.6e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
ro_amplitude = 0.21
ro_dac_length = 0.11e-6
adc.set_nums(1000)
#adc.set_software_nums_multi(1)
set_adc_nop(ro_adc_length)

In [ ]:
#tomography.set_sequence([tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)], awg_channels)
plt.plot(np.mean(np.real(adc.measure()['Voltage']), axis=0))

In [ ]:
plt.pcolormesh(np.real(adc.measure()['Voltage']));
plt.colorbar()

In [ ]:
adc.stop()

### Прохождение считывающих испульсов разной амплитуды

In [ ]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0, 1, 51), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

### Осцилляции Раби

In [ ]:
ex_amplitude = 0.8
pause_length = 16e-9
lengths = np.linspace(0e-9, 2e-6, 201)
readout_begin = np.max(lengths)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length'), filename='Rabi')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

In [ ]:
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

### Осцилляции Рамзея

In [ ]:
ex_amplitude = 0.8
pi2_pulse = 22e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 2000e-9, 101)
target_freq_offset = 17e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
#save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

In [ ]:
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

### Затухание свободной индукции (T1)

In [ ]:
ex_amplitude = 0.8
pi_pulse = 44e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 10000e-9, 41)
readout_begin = np.max(delays)+pi_pulse
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
#save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

In [ ]:
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

### Rabi amplitude oscillations

In [ ]:
ex_amps = np.linspace(0.,1.0,51)
pause_length = 32e-9
length = 32e-9
sigma = 8e-9
num_pulses = 4
readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', pi_pulse, pg.gauss_hd, sigma, ex_amplitude, 0), 
                pg.p(None, readout_begin-length)]*num_pulses+\ 
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

### gaussian $\pi/2$ pulse amplitudes

In [ ]:
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
amp_x = 0.925
length = 32e-9
pause_length = 8e-9
sigma = 8e-9
amp_x_hd = 0.925
length_hd = 32e-9
sigma_hd = 8e-9
pause_length_hd = 8e-9
alpha_hd = -0.92e-9

### Ramsey with gauss pulses

In [ ]:
delays = np.linspace(0e-9, 2000e-9, 201)
target_freq_offset = 20e6

def set_delay(delay):
    sequence = [pg.p(None, readout_begin-length),
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude, 0), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.gauss_hd, sigma, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi), 0), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

# Optimizing readout pulse settings

In [ ]:
ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma), pg.p(None, pause_length)]*2
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc.set_software_nums_multi(20)
adc_sz.repeat_samples = 1

target_min = lambda: 1-adc_sz.measure()['Calibrated fidelity binary']

class ro_pulse_generator:
    def __init__(self, adc_sz, ro_amplitude, ro_dac_length):
        self.ro_amplitude = ro_amplitude
        self.ro_dac_length = ro_dac_length
        self.adc_sz = adc_sz
    def make_ro_seq(self):
        adc_sz.ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', self.ro_dac_length, pg.rect, self.ro_amplitude)]
    def set_ro_length(self, ro_dac_length):
        self.ro_dac_length = ro_dac_length
        self.make_ro_seq()
    def set_ro_amplitude(self, ro_amplitude):
        self.ro_amplitude = ro_amplitude
        self.make_ro_seq()

ro_dac_lengths = np.linspace(0, 0.05e-6, 6)
ro_dac_amplitudes = np.linspace(0.8,1.0, 5)

ro_pulse_generator_inst = ro_pulse_generator(adc_sz, ro_amplitude, ro_dac_length)
solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 3e-8), 
                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.9), maxfun=30)
ro_dac_length = solution[0]
ro_dac_amplitude = solution[1]
#
#sweep.sweep(adc_sz, (ro_dac_lengths, ro_pulse_generator_inst.set_ro_length, 'Readout pulse length'), 
#                    (ro_dac_amplitudes, ro_pulse_generator_inst.set_ro_amplitude, 'Readout pulse amplitude'))

### |0> & |1> readout calibration

In [ ]:
ro_dac_length = 50e-9
ro_dac_amplitude = 0.13
ex_calib_seq = [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]*2
#ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, ro_amplitude)]
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, 1.0)]
adc_sz = tomography_new.sz_measurer(adc, ex_calib_seq, ro_seq, pg)
adc_sz.save_last_samples = True
adc_sz.repeat_samples = 1
adc.set_software_nums_multi(20)
print(adc_sz.measure())
#sz_filter = adc_sz.calibrate()
#print('ROC AUC:', adc_sz.calib_roc_auc, 'Fidelity:', adc_sz.calib_fidelity)
#print('ROC AUC binary:', adc_sz.calib_roc_auc_binary, 'Fidelity binary:', adc_sz.calib_fidelity_binary) 

In [ ]:
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[0,:,:]), vmin=15, vmax=25); plt.colorbar()
plt.figure()
plt.pcolormesh(np.real(adc_sz.samples[1,:,:]), vmin=15, vmax=25); plt.colorbar()

In [ ]:
plt.hist(adc_sz.calib_pred.T, bins=20)

In [ ]:
#plt.scatter(np.real(adc_sz.calib_usl_pred[0,:]), np.imag(adc_sz.calib_usl_pred[0,:]), color='r', s=1)
#plt.scatter(np.real(adc_sz.calib_usl_pred[1,:]), np.imag(adc_sz.calib_usl_pred[1,:]), color='b', s=1)
#plt.figure()
#plt.hist(adc_sz.calib_pred.T, bins=20)
plt.figure()
#plt.hist(np.real(adc_sz.calib_usl_pred.T), bins=20)
plt.plot(np.real(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.imag(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)))
plt.plot(np.real(adc_sz.calib_feature))
plt.plot(np.imag(adc_sz.calib_feature))

### Tomography

In [ ]:
observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Xo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq, 
                 'operator': observables['X']},
            'Yo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['Y']},
            '-Xo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-X']},
            '-Yo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-Y']},
            'Zo': {'pulses':ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'x':{'operator':observables['X']},
                      'y':{'operator':observables['Y']},
                      'z':{'operator':observables['Z']}}
tomo = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [ ]:
proj_seq = {'Z': {'pulses':ro_seq, 'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq)

In [ ]:
## tomography of |0> state
adc.set_software_nums_multi(20)
tomo.set_prepare_seq([])
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*2)
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, alpha_hd), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, alpha_hd), pg.p(None, pause_length)]*10)
tomo.measure()

In [ ]:
## tomography of |1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]+
                     [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma, -0.92e-9), pg.p(None, pause_length),
                     pg.p('iq_ex', length, pg.gauss_hd, 0, amp_x, sigma, -0.92e-9), pg.p(None, pause_length)]*10)
tomo.measure()

In [ ]:
## tomography of |0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of |0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, 1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of -|0>-|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

In [ ]:
## tomography of -|0>+|1> state
tomo.set_prepare_seq([pg.p('iq_ex', length, pg.gauss_hd, 0, -1j*amp_x, sigma), pg.p(None, pause_length)])
tomo.measure()

# Randomized benchmarking

In [ ]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomo)

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}
bench.interleavers = {'X': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]])},
                      'Y': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]])},
                      '-X':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]])},
                      '-Y':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -1j*amp_x, 0, sigma, alpha_hd), pg.p(None, pause_length)],
                            'unitary': np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])},
                      'I':{'pulses':[], 'unitary':np.asarray([[1, 0], [0,1]])}}
bench.sequence_length = 10
bench.random_sequence_num = 6
bench.prepare_random_interleaving_sequences()
#bench.generate_interleaver_sequence_from_names()

In [ ]:
seq_lengths = np.arange(101)
sweep.sweep(bench, (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number'))

In [ ]:
bench.target_gate = bench.interleavers['X']['pulses']
bench.target_gate_unitary = bench.interleavers['X']['unitary']
bench.target_gate_name = 'X (benchmarking)'
tfunc = lambda : np.mean(bench.measure()['Euclidean distance'])
class hd_pulse_generator:
    def __init__(self, pulse_setter, pg, channel):
        self.amp_x = -amp_x_hd
        self.amp_y = amp_x_hd
        self.sigma = sigma_hd
        self.length = length_hd
        self.pause_length = pause_length_hd
        self.alpha = alpha_hd
        
        self.pg = pg
        self.channel = channel
        self.pulse = self.pulse_gen()
        self.pulse_setter = pulse_setter
    def set_real_amplitude(self,x):
        self.amp_x = x
        self.pulse_setter(self.pulse_gen())
    def set_imag_amplitude(self,y):
        self.amp_y = y
        self.pulse_setter(self.pulse_gen())
    def set_sigma(self,s):
        self.sigma = s
        self.pulse_setter(self.pulse_gen())
    def set_length(self,l):
        self.length = l
        self.pulse_setter(self.pulse_gen())
    def set_pause_length(self, p):
        self.pause_length = p
        self.pulse_setter(self.pulse_gen())
    def set_alpha(self, a):
        self.alpha = a
        self.pulse_setter(self.pulse_gen())
    def pulse_gen(self):
        return [pg.p(self.channel, self.length, self.pg.gauss_hd, self.amp_x+1j*self.amp_y, 0, self.sigma, self.alpha), pg.p(None, self.pause_length)]
    
xg = hd_pulse_generator(bench.set_target_pulse, pg, 'iq_ex')

In [ ]:
sol, score = sweep.optimize(tfunc,  (xg.set_real_amplitude, xg.amp_x), 
                                    (xg.set_imag_amplitude, xg.amp_y),
                                    (xg.set_alpha, xg.alpha))

In [ ]:
amps = np.linspace(0.925, 0.975, 21)
alphas = np.linspace(-2e-9, 2e-9, 11)
sweep.sweep(bench, (amps, xg.set_real_amplitude, 'Amplitude'), (alphas, xg.set_alpha, 'HD DRAG coeff'), filename='Randomized interleaved benchmarging X-rotation')

In [ ]:
data = save_pkl.load_pkl('Euclidean distance Randomized interleaved benchmarging X-rotation', 'D:\\qtlab\\ReiData\\data\\2017-07-02\\23-03-34')
plt.figure()
plt.pcolormesh(data[1]['Euclidean distance'][1][1], data[1]['Euclidean distance'][1][0], data[1]['Euclidean distance'][2][:,:,5])
plt.colorbar()

In [ ]:
data[1]

In [ ]:
data[1]['Euclidean distance']

In [ ]:
meas = bench.measure()

In [ ]:
meas['Xo']

In [ ]:
import plotting
imp.reload(plotting)
plotting.plot_measurement(bench.reference_benchmark_result)

# Optimizing Drag

In [ ]:
num_interleved_pulses = 2
alphas = np.linspace(-1e-8, 1e-8, 201)
def set_drag (alpha):
    ramsey_half = [pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                 pg.p(None, pause_length_hd)]
    interleaved_identity = [pg.p('iq_ex', length_hd, pg.gauss_hd, -amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd),
                           pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd)]
    tomo.set_prepare_seq(ramsey_half+interleaved_identity*num_interleved_pulses+ramsey_half)
    
measurement = sweep.sweep(tomo, (alphas, set_drag, 'delta_freq'), filename='DRAG', output=False)

### Randomized Benchmarking

In [ ]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.linspace(0, 100, 11)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [ ]:
rho_m

In [ ]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [ ]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
m

In [ ]:
fidelities

In [ ]:
psis

In [ ]:
rho_m

In [ ]:
rho_t

In [ ]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

In [ ]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

In [ ]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

In [ ]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

In [ ]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

### Optimizing phase between x and y pulses

In [ ]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

In [ ]:
phases[7]

In [ ]:
plt.plot(np.asarray(results).T)

In [ ]:
adc.stop()

In [ ]:
plt.close()

# load calibration data

In [ ]:
import pickle
f0 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-46\\Voltage ro calibration.pkl', 'rb')
f1 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-14\\Voltage ro calibration.pkl', 'rb')
cal0 = pickle.load(f0)
cal1 = pickle.load(f1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt4
cal0_nomean = cal0[1]['Voltage'][2][0,:,:]-np.mean(cal0[1]['Voltage'][2][0,:,:])
cal1_nomean = cal0[1]['Voltage'][2][1,:,:]-np.mean(cal0[1]['Voltage'][2][1,:,:])

cal0_mean = np.mean(cal0_nomean, axis=0)
cal1_mean = np.mean(cal1_nomean, axis=0)

In [ ]:
cal_mean = (cal0_mean+cal1_mean)/2.
cal_diff = (cal1_mean-cal0_mean)

plt.figure('Samples')
plt.plot(np.mean(cal0_nomean, axis=0), label='|0>')
plt.plot(np.mean(cal1_nomean, axis=0), label='|1>')
plt.legend()

plt.figure('Offset & feature')
plt.plot(cal_mean, label='mean')
plt.plot(cal_diff, label='diff')
plt.legend()

In [ ]:
plt.figure('Fourier domain')
plt.plot(np.abs(np.fft.fft(cal0_mean)), label='|0>')
plt.plot(np.abs(np.fft.fft(cal1_mean)), label='|1>') 
plt.plot(np.abs(np.fft.fft(cal_mean)), label='mean')
plt.plot(np.abs(np.fft.fft(cal_diff)), label='diff')
plt.legend()

feature = np.conj(cal_diff/np.sum(np.abs(cal_diff**2)))

In [ ]:
coeffs0 = np.dot(cal0_nomean-cal_mean, feature)
coeffs1 = np.dot(cal1_nomean-cal_mean, feature)
predictions = np.asarray([np.real(coeffs0), np.real(coeffs1)])

In [ ]:
hist_all, bins = np.histogram(predictions, bins='auto')
proba_points = (bins[1:]+bins[:-1])/2.
hists = []
for y in range(2):
    hists.append(np.histogram(predictions[y,:], bins=bins)[0])

hists = np.asarray(hists, dtype=float)
probabilities = hists/hist_all
naive_probabilities = np.asarray([proba_points<0, proba_points>0], dtype=float)
probabilities[np.isnan(probabilities)] = naive_probabilities[np.isnan(probabilities)]
predictor = lambda x: np.interp(x, proba_points, probabilities[1,:], left=0., right=1.)
calib_proba_points = proba_points
calib_proba = probabilities[1,:]
calib_hists = hists

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
roc = roc_curve([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))
roc_auc = roc_auc_score([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))

In [ ]:
plt.figure('ROC')
plt.plot(roc[0], roc[1])
plt.xlabel('True Positive Rate')
plt.ylabel('True Negative Rate')
roc_auc

In [ ]:
plt.figure('probability curve')
plt.plot(calib_proba_points, calib_proba)

In [ ]:
plt.figure('Readout hists')
plt.bar(proba_points-0.25, hists[0], width=0.05, label='|0>')
plt.bar(proba_points+0.25, hists[1], width=0.05, label='|1>')
plt.legend()

In [ ]:
plt.close()

In [ ]:
cal0_nomean.shape

In [ ]:
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal0_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal1_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')

In [ ]:
plt.plot(np.mean(cal0_nomean, axis=0))
plt.plot(np.mean(cal1_nomean, axis=0))